In [1]:
import pandas as pd
from IPython.display import display

In [37]:
# This was for the the original spread
data_master = pd.read_csv('log_forward_problem_Re_variation_short.txt', sep=" ")
# Filter out the 25,50,100
data_master[~data_master.nx.isin([25,50,100])].to_csv('test.txt',index=False, sep=' ')
test =  pd.read_csv('test.txt')
display(test.columns.values)

array(['tFinal Re nx nt uErr pErr runtime'], dtype=object)

In [38]:
analyze_results('test.txt','compiled_test2.xlsx')

array(['tFinal', 'Re', 'nx', 'nt', 'uErr', 'pErr', 'runtime'], dtype=object)

,tFinal,Re,nx,nt,uErr,pErr,runtime,uReduction,pReduction
0,1,1,8,8,0.000909,0.008098,4.67509,NaN,NaN
1,1,1,16,16,0.000128,0.002580,23.03550,7.095900,3.138642
2,1,1,32,32,0.000017,0.001626,217.54400,7.512753,1.586635
3,1,1,64,64,0.000003,0.014545,2515.92000,6.253745,0.111807


,tFinal,Re,nx,nt,uErr,pErr,runtime,uReduction,pReduction
0,1,2,8,8,0.000954,0.004310,4.77417,NaN,NaN
1,1,2,16,16,0.000130,0.001294,23.55710,7.320193,3.330382
2,1,2,32,32,0.000018,0.000902,190.90800,7.435227,1.434514
3,1,2,64,64,0.000003,0.002435,2138.44000,6.922132,0.370516


,tFinal,Re,nx,nt,uErr,pErr,runtime,uReduction,pReduction
0,1,4,8,8,0.001066,0.002457,7.13447,NaN,NaN
1,1,4,16,16,0.000133,0.000651,26.26490,8.009048,3.774124
2,1,4,32,32,0.000019,0.000335,173.44500,7.060347,1.942654
3,1,4,64,64,0.000003,0.002791,1852.92000,5.885597,0.120047


,tFinal,Re,nx,nt,uErr,pErr,runtime,uReduction,pReduction
0,1,8,8,8,0.001406,0.001621,6.2075,NaN,NaN
1,1,8,16,16,0.000140,0.000331,22.1127,10.050314,4.892808
2,1,8,32,32,0.000023,0.000345,158.6440,6.173157,0.961349
3,1,8,64,64,0.000005,0.002898,1585.5800,4.697512,0.118909


,tFinal,Re,nx,nt,uErr,pErr,runtime,uReduction,pReduction
0,1,16,8,8,0.001990,0.001216,6.33962,NaN,NaN
1,1,16,16,16,0.000161,0.000181,21.88410,12.389924,6.736929
2,1,16,32,32,0.000031,0.000130,151.59600,5.117930,1.383482
3,1,16,64,64,0.000008,0.000709,1396.19000,3.982554,0.183918


,tFinal,Re,nx,nt,uErr,pErr,runtime,uReduction,pReduction
0,1,32,8,8,0.002461,0.000918,6.5688,NaN,NaN
1,1,32,16,16,0.000212,0.000129,24.7339,11.598145,7.117555
2,1,32,32,32,0.000047,0.000028,143.8590,4.485847,4.667323
3,1,32,64,64,0.000013,0.000431,1288.8400,3.751541,0.064127


,tFinal,Re,nx,nt,uErr,pErr,runtime,uReduction,pReduction
0,1,64,8,8,0.002686,0.000976,7.54755,NaN,NaN
1,1,64,16,16,0.000333,0.000125,25.78330,8.056236,7.810055
2,1,64,32,32,0.000074,0.000020,138.76300,4.499673,6.329204
3,1,64,64,64,0.000019,0.000312,1218.12000,3.901085,0.063288


,tFinal,Re,nx,nt,uErr,pErr,runtime,uReduction,pReduction
0,1,128,8,8,0.003308,0.001406,9.21494,NaN,NaN
1,1,128,16,16,0.000587,0.000128,28.44790,5.633078,10.999515
2,1,128,32,32,0.000119,0.000017,133.38200,4.936243,7.710666
3,1,128,64,64,0.000026,0.000832,1160.35000,4.529252,0.019915


In [46]:
data = data_master.copy()
data = data[~data['nx'].isin([25,50,100])]
ReDict = {}
writer = pd.ExcelWriter('compiled_test.xlsx')   
for value in data.Re.unique():
    ReDict[value] = data[data['Re'].eq(value)].reset_index(drop=True)
    ReDict[value]['uReduction'] = ReDict[value]['uErr'].shift(1) / ReDict[value]['uErr']
    ReDict[value]['pReduction'] = ReDict[value]['pErr'].shift(1) / ReDict[value]['pErr']
    ReDict[value].to_excel(writer,
                           sheet_name='Re='+str(value),
                           index=False,
                           columns=['Re','nx','nt',
                                    'uErr','uReduction',
                                   'pErr','pReduction',
                                   'runtime'])
    sheet = writer.sheets['Re='+str(value)]
    sheet.set_column('A:H', 15)
    
#     display(ReDict[value])
    

writer.save()

In [35]:
import pandas as pd
def analyze_results(log_file,output_file):
    # Get the data from the formatted csv
    data = pd.read_csv(log_file, sep=' ')
    display(data.columns.values)
    # Create the ExcelWriter to send the information to xlsx
    writer = pd.ExcelWriter(output_file)
    
    # Loop over the choices for ending time (tFinal)
    for time in data.tFinal.unique():
        # Loop over the choices for the Reynolds number (Re)
        for Re in data.Re.unique():
            # Get the information for a single pair of time and Re
            df = data[data.tFinal.eq(time) & data.Re.eq(Re)].reset_index(drop=True)
            # Get the reduction in error from the increase  in spatial/temporal resolution
            df['uReduction'] = df['uErr'].shift(1)/df['uErr']
            df['pReduction'] = df['pErr'].shift(1)/df['pErr']
            # Write results to a sheet in output_file
            sheetname = 'tFinal='+str(time)+', Re='+str(Re)
            df.to_excel(writer,
                       sheet_name=sheetname,
                       index=False,
                       columns=['Re','nx','nt',
                                    'uErr','uReduction',
                                   'pErr','pReduction',
                                   'runtime'])
            sheet = writer.sheets[sheetname]
            sheet.set_column('A:I',15)
            display(df)
    
    writer.save()
    writer.close()

In [36]:
# This go was for Re={10,100}, nx=24,36,48,60,72, tFinal=1,4
log_file = 'log_pd_readable_tFinal1_4_Re_10_100.txt'
output = 'compiled_test.xlsx'
analyze_results(log_file=log_file,output_file=output)

array(['tFinal', 'Re', 'nx', 'nt', 'uErr', 'pErr', 'runtime'], dtype=object)

,tFinal,Re,nx,nt,uErr,pErr,runtime,uReduction,pReduction
0,1,10,24,24,0.000049,0.000123,67.147,NaN,NaN
1,1,10,36,36,0.000019,0.000273,222.542,2.570236,0.451342
2,1,10,48,48,0.000010,0.000946,564.889,1.859790,0.288783
3,1,10,60,60,0.000006,0.002499,1204.290,1.591044,0.378308
4,1,10,72,72,0.000004,0.005222,2222.390,1.444869,0.478580


,tFinal,Re,nx,nt,uErr,pErr,runtime,uReduction,pReduction
0,1,100,24,24,0.000188,0.000038,60.1001,NaN,NaN
1,1,100,36,36,0.000078,0.000110,191.8580,2.422976,0.345628
2,1,100,48,48,0.000043,0.000573,457.9470,1.826585,0.192743
3,1,100,60,60,0.000027,0.001144,951.9260,1.581295,0.500508
4,1,100,72,72,0.000019,0.001126,1705.6100,1.448750,1.016827


,tFinal,Re,nx,nt,uErr,pErr,runtime,uReduction,pReduction
0,4,10,24,96,0.000029,0.000015,268.765,NaN,NaN
1,4,10,36,144,0.000015,0.001075,911.744,1.915652,0.014351
2,4,10,48,192,0.000009,0.003787,2336.940,1.664013,0.283901
3,4,10,60,240,0.000006,0.009998,4891.670,1.510640,0.378759
4,4,10,72,288,0.000004,0.020931,9092.730,1.404308,0.477680


,tFinal,Re,nx,nt,uErr,pErr,runtime,uReduction,pReduction
0,4,100,24,96,0.000186,0.000021,233.162,NaN,NaN
1,4,100,36,144,0.000077,0.000452,770.927,2.407771,0.046850
2,4,100,48,192,0.000042,0.002314,1858.320,1.820977,0.195458
3,4,100,60,240,0.000027,0.004614,3844.650,1.578715,0.501469
4,4,100,72,288,0.000019,0.004521,6968.130,1.447384,1.020639
